# 행렬 분해를 이용한 추천서비스(Matrix Factorization)_특이값 분해

In [2]:
import pandas as pd

#csv파일 dataframe으로 변환하고 읽기
review_data_2 = pd.read_csv('./yogiyo/order_db_v2.csv', encoding='ansi')
                          #encoding='utf-8')
#불필요한 데이터 drop하기
review_data_2.drop('Unnamed: 0', axis=1, inplace=True)

review_data_2#.head(5)

,user_id,order_date,point,comment,order_no,order_item
0,lo,2020년 11월 23일,5,배달도 양도 맛도 쵝오인데 너무맛있는당면 추가있었으면해요,9181,궁중식 순살찜닭
1,ni,2020년 11월 23일,4,무난무난 쉐플리 ㅇㅇ,9180,허니갈릭 닭가슴살 시저샐러드
2,qk,2020년 11월 23일,3,뭔가 고기에 양념이 안 스며든 느낌 ㅠㅠ 양은 많아욥,9179,궁중식 순살찜닭
3,dk,2020년 11월 23일,5,빠른 배달 감사합니다.,9178,두툼한 통등심 가츠동
4,ji,2020년 11월 23일,5,맛있어요 ㅋㅋㅋㅋ 넘넘만족해용,9177,로스트 치킨과 매콤한 크림리조또
...,...,...,...,...,...,...
15290,mo,2017년 11월 03일,5,건강하게 한끼 먹어서 좋았습니다. 샐러드 양도많고 퀄리티도 생각했던거 보다 좋네요....,2,허니갈릭 닭가슴살 시저샐러드
15291,mo,2017년 11월 03일,5,건강하게 한끼 먹어서 좋았습니다. 샐러드 양도많고 퀄리티도 생각했던거 보다 좋네요....,2,코카콜라 제로
15292,b9,2017년 11월 02일,5,일하면서 먹어서 사진을 이쁘게 찍진 못했네요 ㅜㅜ. 일단 맛은 최고! 배달도 친절!...,1,모둠나물 제육비빔밥
15293,b9,2017년 11월 02일,5,일하면서 먹어서 사진을 이쁘게 찍진 못했네요 ㅜㅜ. 일단 맛은 최고! 배달도 친절!...,1,순살 새우치킨 콤보 (양념)


In [115]:
import pandas as pd

#csv파일 dataframe으로 변환하고 읽기
menu_data = pd.read_csv('./yogiyo/Menu_DB.csv', encoding='ansi')
                          #encoding='utf-8')

menu_data

,menu_name,menu_categori,price,menu_exp
0,씨푸드 나시고랭,메인,10000,큼지막한 새우와 다양한 해산물이 쏙쏙! 이국적인 맛이 고스란히 담긴 인도네시아풍 볶음밥
1,그라나파다노 김치볶음밥,메인,10000,트러플향 가득! 르꼬르동블루 출신 원영호 셰프의 레시피로 만든 프리미엄 김치볶음밥
2,비빔명란 가라아케동,메인,10000,촉촉한 가라아게와 톡톡 터지는 명란이 더해진 근사한 별미 (명란을 잘 섞어드세요)
3,매콤대패삼겹덮밥,메인,11000,고소한 대패삼겹살에 특제 양념의 매콤달콤함을 더한 셰플리키친 시그니처 메뉴
4,모둠나물 제육비빔밥,메인,11000,매콤달콤한 제육불고기와 여섯 종류의 모둠나물이 조화롭게 어우러진 든든한 한 끼
5,소불고기 비빔밥,메인,11000,전 세계인이 사랑하는 한식의 대표주자 비빔밥과 불고기의 환상적 만남
6,궁중식 순살찜닭,메인,11000,면역력에 좋은 귀한재료가 아낌없이 듬뿍! 임금님도 반할 만한 궁중식 순살찜닭
7,두툼한 통등심 가츠동,메인,10000,육즙 가득 두툼한 생등심이 통으로! 남녀노소가 사랑하는 셰플리의 스터디셀러
8,대패삼겹덮밥,메인,11000,미쉐린 등재 레스토랑 '민스키친'의 김민지 셰프가 선보이는 든든한 한 끼
9,로스트 치킨과 매콤한 크림리조또,메인,11000,허브향 닭다리살이 통으로! 청양고추로 느끼함을 잡은 고급스러운 크림 리조또


In [67]:
#menu_data, review_data_2 merge 시키기
merge_data = pd.merge(review_data_2, menu_data, left_on='order_item', 
                      right_on='menu_name', how='outer')
merge_data.drop(['menu_name','menu_exp'], axis=1, inplace=True)
merge_data#.head(35)

,user_id,order_date,point,comment,order_no,order_item,menu_categori,price
0,lo,2020년 11월 23일,5,배달도 양도 맛도 쵝오인데 너무맛있는당면 추가있었으면해요,9181,궁중식 순살찜닭,메인,11000
1,qk,2020년 11월 23일,3,뭔가 고기에 양념이 안 스며든 느낌 ㅠㅠ 양은 많아욥,9179,궁중식 순살찜닭,메인,11000
2,ji,2020년 11월 23일,5,참 맛있고 배달 빨라요,9175,궁중식 순살찜닭,메인,11000
3,co,2020년 11월 23일,5,잘먹었습니다 고기냄새도안나고 아주 좋아요,9171,궁중식 순살찜닭,메인,11000
4,ja,2020년 11월 23일,3,배달원은 무뚝뚝 음식만 띡 주고가서 약간 불친절 음식은 뭐 식당에서 먹는 찜닭이랑 ...,9163,궁중식 순살찜닭,메인,11000
...,...,...,...,...,...,...,...,...
15290,rh,2018년 02월 12일,5,배달음식에서 맛보기 드문 훌륭한 맛입이다. 담백하게 맛있어요. 야채를 먹지않는 아들...,315,페리에 레몬탄산수,음료,2500
15291,ma,2018년 02월 07일,5,매번 주문해서 먹을때마다 만족합니다~^^,294,페리에 레몬탄산수,음료,2500
15292,jg,2018년 01월 20일,4,굳 맛있어용~~~ㅋㅋ,211,페리에 레몬탄산수,음료,2500
15293,gg,2017년 12월 18일,5,궁중 비빔밥 맛있어요 양도 많고 제가 딱 원하던맛!! 자주 시켜먹을듯해요,119,페리에 레몬탄산수,음료,2500


In [68]:
#사용자별 각 메뉴에대한 맛평가 데이터 만들기
user_menu_rating = merge_data.pivot_table(
    'point', 
    index = 'user_id', 
    columns = 'order_item').fillna(0)
user_menu_rating.tail()

order_item,계란말이,궁중식 순살찜닭,그라나파다노 김치볶음밥,대패삼겹덮밥,된장국추가,두툼한 통등심 가츠동,로스트 치킨과 매콤한 크림리조또,매콤대패삼겹덮밥,멘보샤,명란마요 가라아게,...,스프라이트,씨푸드 나시고랭,어향가지 덮밥,어향가지 덮밥&멘보샤,종가집김치,코카콜라,코카콜라 제로,트러플 감자튀김,페리에 레몬탄산수,허니갈릭 닭가슴살 시저샐러드
user_id,,,,,,,,,,,,,,,,,,,,,
zm,0.0,0.0,0.000000,0.000000,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zn,0.0,4.0,0.000000,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zo,0.0,4.0,4.000000,4.714286,0.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zx,0.0,5.0,0.000000,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,5.0
zz,0.0,5.0,4.857143,4.833333,0.0,4.0,4.5,0.0,5.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
import numpy as np

#matrix는 위에서만든 pivot_table값을 numpy matrix로 만듬
matrix = user_menu_rating.values

#user_rating_avg는 사용자의 평균 맛평가
user_rating_avg = np.mean(matrix, axis = 1)

#matrix_user_avg : 사용자-평가에대해 사용자 평균 평점을 뺀것
matrix_user_avg = matrix - user_rating_avg.reshape(-1, 1)

matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 4.        , 0.        ,
        0.        ],
       ...,
       [0.        , 4.        , 4.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 5.        , 0.        , ..., 0.        , 0.        ,
        5.        ],
       [0.        , 5.        , 4.85714286, ..., 0.        , 0.        ,
        0.        ]])

In [70]:
#614명의 사용자가 평가한 점수의 평균데이터
pd.DataFrame(matrix_user_avg, columns=user_menu_rating.columns).head()
#pd.DataFrame(matrix_user_avg, index=user_menu_rating.index, columns=user_menu_rating.columns)

order_item,계란말이,궁중식 순살찜닭,그라나파다노 김치볶음밥,대패삼겹덮밥,된장국추가,두툼한 통등심 가츠동,로스트 치킨과 매콤한 크림리조또,매콤대패삼겹덮밥,멘보샤,명란마요 가라아게,...,스프라이트,씨푸드 나시고랭,어향가지 덮밥,어향가지 덮밥&멘보샤,종가집김치,코카콜라,코카콜라 제로,트러플 감자튀김,페리에 레몬탄산수,허니갈릭 닭가슴살 시저샐러드
0,-0.625000,-0.625000,-0.625000,4.375000,-0.625000,4.375000,4.375000,-0.625000,-0.625000,-0.625000,...,-0.625000,4.375000,-0.625000,-0.625000,-0.625000,-0.625000,-0.625000,-0.625000,-0.625000,-0.625000
1,-0.156250,-0.156250,-0.156250,-0.156250,-0.156250,4.843750,-0.156250,-0.156250,-0.156250,-0.156250,...,-0.156250,-0.156250,-0.156250,-0.156250,-0.156250,-0.156250,-0.156250,-0.156250,-0.156250,-0.156250
2,-0.656250,-0.656250,-0.656250,3.343750,-0.656250,-0.656250,3.343750,-0.656250,-0.656250,-0.656250,...,-0.656250,-0.656250,-0.656250,-0.656250,-0.656250,-0.656250,-0.656250,3.343750,-0.656250,-0.656250
3,-0.984375,2.015625,4.015625,4.015625,0.015625,4.015625,2.515625,-0.984375,-0.984375,-0.984375,...,-0.984375,-0.984375,-0.984375,-0.984375,-0.984375,-0.984375,-0.984375,0.015625,-0.984375,4.015625
4,3.000000,-2.000000,3.000000,3.000000,-2.000000,3.000000,3.000000,-2.000000,3.000000,3.000000,...,-2.000000,3.000000,-2.000000,-2.000000,3.000000,3.000000,-2.000000,-2.000000,-2.000000,-2.000000


In [71]:
from scipy.sparse.linalg import svds
#scipy에서제공하는 svd
#U행렬, sigma행렬, V전치행렬 반환
U, sigma, Vt = svds(matrix_user_avg, k=2)

print(U.shape)
print(sigma.shape)
print(Vt.shape)
#sigma 행렬은 0이 포함되지 않는 값으로만 되어있음

(614, 2)
(2,)
(2, 32)


In [72]:
#위에 sigma행렬을 대칭행렬로 변환
sigma = np.diag(sigma)
print(sigma.shape)
print(sigma[0])
print(sigma[1])

(2, 2)
[52.79370383  0.        ]
[  0.         115.46463575]


In [73]:
#matrix_user_avg를 svd를 적용해 분해한 상태이므로 원복 필요
svd_user_predicted_rating = np.dot(np.dot(U, sigma), Vt) + user_rating_avg.reshape(-1,1)


In [74]:
svd_preds = pd.DataFrame(svd_user_predicted_rating, 
                         index=user_menu_rating.index, 
                         columns=user_menu_rating.columns)
svd_preds

order_item,계란말이,궁중식 순살찜닭,그라나파다노 김치볶음밥,대패삼겹덮밥,된장국추가,두툼한 통등심 가츠동,로스트 치킨과 매콤한 크림리조또,매콤대패삼겹덮밥,멘보샤,명란마요 가라아게,...,스프라이트,씨푸드 나시고랭,어향가지 덮밥,어향가지 덮밥&멘보샤,종가집김치,코카콜라,코카콜라 제로,트러플 감자튀김,페리에 레몬탄산수,허니갈릭 닭가슴살 시저샐러드
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.216336,0.196186,1.997238,3.222695,0.371573,1.846450,2.350563,1.093550,0.201608,0.685321,...,0.965346,0.694999,-0.220565,-0.337136,0.041144,1.190720,-0.114419,1.385945,-0.357598,1.614206
0_,0.288819,1.017689,0.399709,0.586620,0.413448,1.006656,0.276870,0.138946,-0.154799,0.430459,...,0.044668,-0.023696,-0.038674,-0.008689,0.134422,0.090692,-0.017006,0.058979,-0.336488,0.318225
0f,0.471195,0.812217,1.611436,2.449663,0.626315,1.813657,1.767446,0.933714,0.247169,0.825971,...,0.809799,0.611727,0.045902,-0.010680,0.289472,0.969604,0.121163,1.073333,-0.182062,1.338258
1,0.913051,2.397923,2.642769,4.060915,1.300709,3.766292,2.687493,1.343687,-0.013453,1.600239,...,1.041355,0.672598,-0.130048,-0.166528,0.447230,1.326170,0.004076,1.425389,-0.859217,2.151989
10,1.404768,1.073704,3.711081,5.251879,1.557785,3.258367,4.229144,2.626198,1.570609,1.965150,...,2.496216,2.167633,0.964399,0.797880,1.237824,2.774718,1.095571,3.045864,0.907768,3.239118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zm,0.011379,-0.559336,0.793711,1.246925,-0.008896,0.253174,1.060383,0.554097,0.345587,0.130873,...,0.564186,0.485080,0.050538,-0.029331,0.044839,0.638572,0.085579,0.758092,0.212956,0.669800
zn,0.727200,2.310575,0.685254,0.896186,0.972780,2.025014,0.362922,0.263198,-0.232224,0.959919,...,0.079131,-0.026218,0.086467,0.169956,0.420244,0.143180,0.116614,0.043473,-0.537465,0.569326
zo,1.487785,1.989013,2.668350,3.558664,1.671716,3.002781,2.800488,1.925167,1.166009,1.877922,...,1.779360,1.563828,0.986520,0.935493,1.270519,1.951308,1.067498,2.049804,0.691797,2.373817


In [111]:
def recommend_menu(svd_preds, og_user_id, og_order_item, og_point, num_recommendations=5):
    
    #맛평가 평점이 높은 순으로 정렬
    sort_user_predictions = svd_preds.index.sort_values(ascending=False)
    
    #원본데이터에서 user_id에 해당하는 데이터 뽑기
    user_data = og_point[og_point.user_id == og_user_id]
    
    #원본데이터에서 사용자가 주문했던 음식은 제외한 데이터 추출
    recommendations = og_order_item[-og_order_item['order_item'].isin(user_data['order_item'])]  
    #사용자의 메뉴 평점이 높은 순으로 정렬된 데이터와 recommendations를 합친다
    recommendations = recommendations.merge(pd.DataFrame(sort_user_predictions).reset_index(), on = 'user_id')
    #컬럼이름을 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {'index' : 'Predictions'}).sort_values('Predictions', ascending=False)
    
    return user_data, recommendations

In [112]:
already_rated, predictions = recommend_menu(svd_preds, 'dk', merge_data, 
                                            merge_data, 10)
already_rated = already_rated.sort_values(by='order_no', ascending=False)
#predictions = predictions.sort_values(by='order_no', ascending=False)

In [113]:
already_rated

,user_id,order_date,point,comment,order_no,order_item,menu_categori,price
1635,dk,2020년 11월 23일,5,빠른 배달 감사합니다.,9178,두툼한 통등심 가츠동,메인,10000
1644,dk,2020년 11월 22일,5,최고였습니다. 또 시킬께요!,9130,두툼한 통등심 가츠동,메인,10000
5990,dk,2020년 10월 27일,5,맛있어요!! 또 시켜먹을거같아요,8866,대패삼겹덮밥,메인,11000
4444,dk,2020년 10월 26일,5,새로운 볶음밥에 감탄했어요~ 단백질양이 부족해보일땐 사이드로 추가주문도 할 수 있어...,8861,계란말이,사이드,2500
11179,dk,2020년 10월 26일,5,새로운 볶음밥에 감탄했어요~ 단백질양이 부족해보일땐 사이드로 추가주문도 할 수 있어...,8861,그라나파다노 김치볶음밥,메인,10000
8310,dk,2020년 10월 26일,5,새로운 볶음밥에 감탄했어요~ 단백질양이 부족해보일땐 사이드로 추가주문도 할 수 있어...,8861,명란마요 가라아게,사이드,5000
11192,dk,2020년 10월 10일,5,오랜만에 시켰는데 맛있네요,8687,그라나파다노 김치볶음밥,메인,10000
3464,dk,2020년 09월 07일,5,맛있는데 약간 텍스쳐가 구림,8304,로스트 치킨과 매콤한 크림리조또,메인,11000
1771,dk,2020년 09월 07일,4,할인 받아서 맛있게 잘 먹었습니다~,8303,두툼한 통등심 가츠동,메인,10000
1832,dk,2020년 08월 13일,1,서초점보다 질기고 냄새도 많이 났습니다. 잘 먹었습니다.,7949,두툼한 통등심 가츠동,메인,10000


In [114]:
predictions#.head(10)

,user_id,order_date,point,comment,order_no,order_item,menu_categori,price,Predictions
2439,0,2020년 10월 08일,5,맛있게 잘먹었습니다,8677,씨푸드 나시고랭,메인,10000,613
2634,0f,2020년 05월 01일,4,맛있게잘먹었습니다 대신 카라아게동인데 바삭한느낌은아니고 튀김옷을입은 오야꼬동에더 가...,6939,트러플 감자튀김,사이드,4500,611
1744,1,2019년 08월 18일,5,맛있엇습니다~~~//~,4069,허니갈릭 닭가슴살 시저샐러드,메인,11000,610
1745,1,2019년 07월 19일,5,양도 많고 맛있어요 !!,3623,허니갈릭 닭가슴살 시저샐러드,메인,11000,610
1746,1,2019년 07월 18일,5,맛있게 잘먹었습니다 ㅎㅎ~~,3600,허니갈릭 닭가슴살 시저샐러드,메인,11000,610
...,...,...,...,...,...,...,...,...,...
1813,zx,2019년 08월 01일,5,맛은 있는데 느려요 ㅋㅋㅋㅋ큐ㅠㅠ,3759,허니갈릭 닭가슴살 시저샐러드,메인,11000,1
1814,zx,2019년 07월 26일,5,맛은 맛잇는데...배달이 너무 좀 느리네용,3710,허니갈릭 닭가슴살 시저샐러드,메인,11000,1
2510,zz,2020년 08월 17일,5,할라피뇨 많이 요청 들어주셔서 감사해요 맛있어요 리조또 치즈맛 강해요 <U+0001...,7977,멘보샤,사이드,6000,0
2509,zz,2018년 07월 07일,4,모든메뉴 다 맛있습니다. 양이 좀 작은 것 같구요.,962,씨푸드 나시고랭,메인,10000,0
